In [ ]:
%pylab inline
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection
import seaborn as sns
sns.set_style("white")

In [ ]:
pfc=load("D1/pfc.npy")
Vs=load("D1/Vdynamics.npy")

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.grid(False)
verts = []
zs = range(5,50,2)
for i in zs:
    verts.append(list(zip(pfc,Vs[i])))    
poly = LineCollection(verts)
ax.add_collection3d(poly, zs=zs, zdir='y')
ax.set_xlim3d(pfc.min(),pfc.max())
ax.set_zlim3d(Vs[-1].min(), Vs[-1].max())
ax.set_ylim3d(asarray(zs).min(), asarray(zs).max())
ax.set_xlabel('Position (cm)',fontsize=20)
ax.set_ylabel('Time (a.u.)',fontsize=20)
ax.set_zlabel('Rate (a.u)',fontsize=20)
plt.show()

# MULTIMAPS

In [ ]:
pfc=load("8Maps_gammas/pfc_0.1.npy")
Vs=load("8Maps_gammas/Vdynamics_0.1.npy")
#Vs_1=load("8Maps0/Vdynamics_1.npy")

In [ ]:
## 2 mappe - plot delle attività ##
for i in range(len(Vs)):
    figure(figsize=(10,5))
    subplot(1,2,1)
    scatter(pfc[0],Vs[i],c="k")
    xlabel("X Map 1",fontsize=15)
    ylabel("activity (a.u)",fontsize=15)
    ylim((-0.5,10))
    subplot(1,2,2)
    scatter(pfc[1],Vs[i],c="k")
    xlabel("X Map 2",fontsize=15)
    ylim((-0.5,10))
    tight_layout()
    savefig('videotemp/frame'+str(i+1)+'.png')

In [ ]:
## m mappe - plot delle attività ##
for i in range(len(Vs)):
    figure(figsize=(20,10))
    for p in range(len(pfc)):
        subplot(2,4,p+1)
        scatter(pfc[p],Vs[i],c="k")
        xlabel("X Map"+str(p+1))
        ylabel("activity (a.u)")
        ylim((-0.5,10))
        tight_layout()
    savefig("videotemp/frame"+str(i+1)+".png")

In [ ]:
def K(x1,x2,L):
        d=x1-x2
        if d>float(L)/2.0:
            d=d-L
        elif d<-float(L)/2.0:
            d=d+L
        return np.exp(-abs(d))
    
def overlap(V,pos):
    m=0
    for i in range(len(V)):
        for j in range(len(V)):
            m=m+V[i]*V[j]*K(pos[i],pos[j],10.0)
    m=m/pow(len(V),2)
    return m
            
def centerOfMass(V,pos):
    ro=0
    for i in range(len(V)):
        ro=ro+V[i]*pos[i]
    ro=ro/float(len(V))
    return ro

def peakPosition(V,pos):
    peak=pos[argmax(V)]
    return peak

In [ ]:
time=range(len(Vs))
m1=[]
m2=[]
ro1=[]
ro2=[]
for i in range(len(Vs)):
    m1.append(overlap(Vs[i],pfc[0]))
    m2.append(overlap(Vs[i],pfc[1]))
    ro1.append(centerOfMass(Vs[i],pfc[0]))
    ro2.append(centerOfMass(Vs[i],pfc[1]))

In [ ]:
peak1=[]
peak2=[]
for i in range(len(Vs)):
    peak1.append(peakPosition(Vs[i],pfc[0]))
    peak2.append(peakPosition(Vs[i],pfc[1]))

In [ ]:
figure(figsize=(15,5))
subplot(1,3,1)
plot(time,m1)
plot(time,m2)
xlabel("time (steps)",fontsize=15)
ylabel("overlap",fontsize=15)
subplot(1,3,2)
plot(time,ro1)
plot(time,ro2)
xlabel("time (steps)",fontsize=15)
ylabel("center of mass",fontsize=15)
subplot(1,3,3)
plot(time,peak1)
plot(time,peak2)
xlabel("time (steps)",fontsize=15)
ylabel("Peak position",fontsize=15)

# MULTIGAMMAS

In [ ]:
gammas=[0.,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
pfc = np.zeros((11,8,1000))
Vs = np.zeros((11,200,1000))
print(gammas)

In [ ]:
for g in range(len(gammas)):
    pfc[g]=load("8Maps_gammas/pfc_"+str(gammas[g])+".npy")
    Vs[g]=load("8Maps_gammas/Vdynamics_"+str(gammas[g])+".npy")
print(pfc.shape)
print(Vs.shape)

In [ ]:
for g in range(len(gammas)):
    for i in range(len(Vs[g])):
        figure(figsize=(20,10))
        for p in range(len(pfc[g])):
            subplot(2,5,p+1)    #10 figure
            scatter(pfc[g][p],Vs[g][i],c="k")
            xlabel("X Map"+str(p+1))
            ylabel("activity (a.u)")
            ylim((-0.5,10))
            tight_layout()
        savefig("vid_gammas_"+str(0.1*g)+"/frame"+str(i+1)+".png")

## OVERLAP FOR M MAPS

In [ ]:
def KS(x1,x2,L):
        d=x1-x2
        if d>float(L)/2.0:
            d=d-L
        elif d<-float(L)/2.0:
            d=d+L
        return np.exp(-abs(d))
    
def overlap(V,pos):
    m=0
    for i in range(len(V)):
        for j in range(i):
            m=m+V[i]*V[j]*KS(pos[i],pos[j],10.0)
    m=m/(float(len(V)*(len(V)-1)/2))
    return m

In [ ]:
m=np.zeros((len(pfc), len(Vs)))
print(m.shape)
print(Vs.shape)
print(pfc.shape)

### Overlap over time

#### single gamma

In [ ]:
pfc=load("8Maps_gammas/pfc_0.2.npy")
Vs=load("8Maps_gammas/Vdynamics_0.2.npy")

In [ ]:
time=range(len(Vs))
m=np.zeros((len(pfc), len(Vs)))
for k in range(len(pfc)):
    for i in time:
        m[k][i]=overlap(Vs[i],pfc[k])

In [ ]:
figure(figsize=(15,5))
for k in range(len(pfc)):
    plot(time,m[k], label="Map "+str(k+1))
    
xlabel("time (steps)",fontsize=15)
ylabel("overlap [$\gamma$'=0.2]",fontsize=15)
plt.legend()

#### loop for  multiple gammas

In [ ]:
gammas=[0.,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
pfc = np.zeros((11,8,1000))
Vs = np.zeros((11,200,1000))
print(gammas)

In [ ]:
m=np.zeros((len(pfc), len(pfc[0]), len(Vs[0])))
for g in range(len(gammas)):
    pfc[g]=load("8Maps_gammas/pfc_"+str(gammas[g])+".npy")
    Vs[g]=load("8Maps_gammas/Vdynamics_"+str(gammas[g])+".npy")
print(pfc.shape)
print(Vs.shape)
print(m.shape)

In [ ]:
time=range(len(Vs[0]))
for r in range(len(pfc)): #scorro sulle 11 gamma
    for k in range(len(pfc[0])): #scorro sulle 8 mappe
        for i in time: #scorro sui 200 step
            m[r][k][i]=overlap(Vs[r][i],pfc[r][k])
    figure(figsize=(15,5))
    for k in range(len(pfc[0])):
        plot(time,m[r][k], label="Map "+str(k+1))
    xlabel("time (steps)",fontsize=15)
    ylabel("overlap",fontsize=15)
    plt.legend()
    plt.suptitle("$\gamma$'= "+str(gammas[r]), fontsize=19)
    savefig("overlap_over_time_"+str(gammas[r])+".png")


### Overlap over gamma
Overlap dell'ultimo time-step per ogni mappa, per ogni gamma

In [ ]:
gammas=np.logspace(-2, log10(20), 10)
pfc = np.zeros((len(gammas),8,1000))
Vs = np.zeros((len(gammas),200,1000))
print(gammas)

In [ ]:
m_g=np.zeros((len(gammas),len(pfc[0])))
for g in range(len(gammas)):
    pfc[g]=load("8Maps_gammas/pfc_"+str(g)+".npy")
    Vs[g]=load("8Maps_gammas/Vdynamics_"+str(g)+".npy")
print(pfc.shape)
print(Vs.shape)
print(m_g.shape)

In [ ]:
for k in range(len(gammas)): #scorro sulle 11 gamma
    for r in range(len(pfc[0])): #scorro sulle 8 mappe
        m_g[k][r]=overlap(Vs[k][len(Vs[0])-1],pfc[k][r])

max_overlap=[]
for k in range(len(gammas)):
    max_overlap.append(amax(m_g[k]))
print(max_overlap)

In [ ]:
figure(figsize=(15,5))
for k in range(len(pfc)):
    plot(log10(gammas),max_overlap, 'o-')
xlabel("gammas",fontsize=15)
ylabel("last overlap",fontsize=15)
savefig("overlap_over_gamma.png")

In [ ]:
p=np.logspace(-2, log10(20), 10)
print(p)

# 2D

In [ ]:
pfc=load("D2/pfc_7.npy")
Vs=load("D2/Vdynamics_7.npy")
print(len(Vs))
print(len(pfc))
print(Vs.shape)
print(pfc.shape)

In [ ]:
## 2 MAPS only ##
# for i in range(len(Vs)):
#     figure(figsize=(10,5))
#     subplot(1,2,1)
#     scatter(pfc[0].T[0],pfc[0].T[1],c=Vs[i],cmap=cm.jet)
#     subplot(1,2,2)
#     scatter(pfc[1].T[0],pfc[1].T[1],c=Vs[i],cmap=cm.jet)
#     tight_layout()
#     savefig("2Dvideotemp/frame"+str(i+1)+".png")

In [ ]:
for i in range(150): #len(Vs)-> in realtá Vs contiene 200 timesteps 
    figure(figsize=(20,4))
    for p in range(len(pfc)):
        subplot(1,5,p+1)    #10 figure
        scatter(pfc[p].T[0],pfc[p].T[1],c=Vs[i],cmap=cm.jet)
        xlabel("X Map"+str(p+1))
        tight_layout()
        savefig("2Dvideotemp/frame"+str(i+1)+".png")

In [ ]:
random.shuffle(pfc)

In [ ]:
scatter(pfc[0].T[0],pfc[0].T[1])

In [ ]:
pfc